In [1]:
from __future__ import print_function
import os
#data_path = ['data']

In [2]:
import numpy as np
import pandas as pd

#filepath = os.sep.join(data_path + ['Ames_Housing_Sales.csv'])
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
y_test = test['SalePrice']
test = test.drop('SalePrice',axis=1)

In [4]:
y_train = train['SalePrice']
train = train.drop('SalePrice',axis=1)

In [5]:
ntrain = train.shape[0]
#ntest = test.shape[0]

data = pd.concat((train, test)).reset_index(drop=True)
#data.drop(['SalePrice'], axis=1, inplace=True)

In [6]:
# Feature engineering
data['TotalSF'] = data['1stFlrSF'] + data['2ndFlrSF'] + data['TotalBsmtSF']
data['YrDiff'] = data['YrSold'] - data['YearRemodAdd']

In [7]:
data.shape

(1379, 81)

In [8]:
col = data.dtypes != np.object
check = data.columns[col]
# convert 0 value to average
# for i in range(len(check)): 
#     data[check[i]] = data[check[i]].apply(lambda x: np.nan if x == 0 else x)

In [9]:
# OneHot Encoding
mask = data.dtypes == np.object
categorical_cols = data.columns[mask]

In [10]:
num_ohc_cols = (data[categorical_cols]
                .apply(lambda x: x.nunique())
                .sort_values(ascending=False))

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Copy of the data
data_ohc = data.copy()

# The encoders
le = LabelEncoder()
ohc = OneHotEncoder()

for col in num_ohc_cols.index:
    
    # Integer encode the string categories
    dat = le.fit_transform(data_ohc[col]).astype(np.int)
    
    # Remove the original column from the dataframe
    data_ohc = data_ohc.drop(col, axis=1)

    # One hot encode the data--this returns a sparse array
    new_dat = ohc.fit_transform(dat.reshape(-1,1))

    # Create unique column names
    n_cols = new_dat.shape[1]
    col_names = ['_'.join([col, str(x)]) for x in range(n_cols)]

    # Create the new dataframe
    new_df = pd.DataFrame(new_dat.toarray(), 
                          index=data_ohc.index, 
                          columns=col_names)

    # Append the new data to the dataframe
    data_ohc = pd.concat([data_ohc, new_df], axis=1)

In [11]:
data_ = data_ohc

In [12]:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# data_ = imputer.fit_transform(data_ohc)

In [13]:
data_.shape

(1379, 296)

In [14]:
# # NN-imputation
# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors=3, weights="uniform")
# data_ = imputer.fit_transform(data_ohc)

In [15]:
# Data scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_mms = scaler.fit_transform(data_)

In [16]:
data_mms.shape

(1379, 296)

In [17]:
y_train.shape

(1199,)

In [18]:
X_train = data_mms[:ntrain]
X_test = data_mms[ntrain:]

In [19]:
from sklearn.feature_selection import SelectPercentile
select = SelectPercentile(percentile=80) # 60프로 선택
select.fit_transform(X_train, y_train)    # 초기 데이터로 특성 선택
data_ = select.transform(data_mms)
#X_test_selected = select.transform(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [ 67  72  82  85  95 100 119 123 133 169 179 199 235] are constant.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [20]:
X_train = data_[:ntrain]
X_test = data_[ntrain:]

In [21]:
# Feature selection, Scaling
from sklearn.decomposition import PCA

pca = PCA(n_components=236)
pca.fit(data_)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [22]:
from sklearn import linear_model

y_train_log = np.log1p(y_train)
model = linear_model.LinearRegression()
model.fit(X_train, y_train_log)     

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [23]:
pred = model.predict(X_test)
y_pred = np.expm1(pred)

In [30]:
from sklearn.metrics import mean_squared_error

# y_true = np.expm1(list(y_test))
#mean_squared_error(y_data, y_pred)  

# Root mean squared error
mse = mean_squared_error(y_test, y_pred)  
rmse = np.sqrt(mse)
rmse

144014.89007711827

In [25]:
np.savetxt('result.txt', y_pred, delimiter=',',  fmt = "%s")